In [1]:
import sys
sys.path.append('../')
import os
from data_partitioner_jsonl import *
from lisa_tagging_reddit import *

INFO:numexpr.utils:Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
file_name = 'corpus.jsonl'

In [13]:
input_path = os.path.join('/shared/3/projects/hiatus/idiolect/data/full_pilot/long-reddit', file_name)
partition_dir = os.path.join('/shared/3/projects/hiatus/tagged_data/partitions/long-reddit', file_name.split('.')[0]) 
tag_dir = os.path.join('/shared/3/projects/hiatus/tagged_data/partitions/tagged-long-reddit', file_name.split('.')[0])
output_dir= '/shared/3/projects/hiatus/tagged_data/idiolect/long-reddit'
    
# ensures all directories will exist
for directory in [partition_dir, tag_dir, output_dir]:
    os.makedirs(directory, exist_ok=True)

# initialize author subreddit dictionary
author_subreddit_counts = {}


## Partition the file into 100 parts

In [4]:
%%time
partition_file(input_path, partition_dir, chunks=50)

## Tag each partition with 1 CPU 

Set nice value low so we don't hog the server

**Tagger config**

In [5]:
config = load_config()
config.update({'use_gpu': False, 
               'biber': True,
               'binary_tags': True, 
               'function_words': True,
               'token_normalization': 100})
tokenizer = load_tokenizer(use_gpu=False)
config

{'biber': True,
 'function_words': True,
 'binary_tags': True,
 'function_words_list': [],
 'token_normalization': 100,
 'use_gpu': False,
 'show_progress': False,
 'n_processes': 1,
 'processing_size': 10000,
 'return_text': False,
 'drop_last_batch_pct': 0.5}

In [6]:
partition_dir

'/shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus'

In [7]:
%%time
import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

tag_partitions(config,
               input_directory=partition_dir,
               output_directory=tag_dir,
               num_workers=51,
               post_counts=author_subreddit_counts,
               default_niceness=0
               )

/shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-1.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-3.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-2.jsonl


Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-7.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-11.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-13.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-5.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-9.jsonl
Tagging file /shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/partition-4.jsonl
Tagging file /shared/3/projects/hiatu

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [11]:
# utilities:

# Counts the number of tagged entries across each partition
# from collections import defaultdict
# counts = defaultdict(int)
# for i in range(1, 103):
#     try:
#         counts[count_lines(tag_dir + f"/partition-{i}.jsonl-tagged.gz")] += 1
#     except Exception:
#         continue
# print(counts)

def print_stats(post_counts):
    total_posts = 0
    authors_over_5 = 0
    authors_over_10 = 0
    unique_subs = set()

    for author, subs in post_counts.items():
        author_posts = sum(subs.values())
        total_posts += author_posts
        unique_subs.update(subs.keys())
        
        if author_posts > 5:
            authors_over_5 += 1
        if author_posts > 10:
            authors_over_10 += 1

    total_authors = len(post_counts)
    avg_posts = float(total_posts) / total_authors if total_authors else 0.0

    print(f"Total number of authors: {total_authors}")
    print(f"Total number of posts: {total_posts}")
    print(f"Average number of posts per author: {avg_posts:.2f}")
    print(f"Total number of authors with over 5 total posts: {authors_over_5}")
    print(f"Total number of authors with over 10 total posts: {authors_over_10}")
    print(f"Total number of unique subreddits: {len(unique_subs)}")
    
# Individual partition
print_stats(author_subreddit_counts) 

# #All partitions
# import csv
# import os
# from collections import defaultdict

# def read_tsv_to_dict(file_path):
#     # Function to read a single TSV and update the main dictionary
#     with open(file_path, 'r', newline='') as f:
#         reader = csv.DictReader(f, delimiter='\t')
#         for row in reader:
#             author = row['author']
#             subreddit = row['subreddit']
#             post_count = int(row['post_count'])
#             # If author exists in the dictionary, update their count for the subreddit
#             if author in aggregated_data:
#                 if subreddit in aggregated_data[author]:
#                     aggregated_data[author][subreddit] += post_count
#                 else:
#                     aggregated_data[author][subreddit] = post_count
#             else:
#                 # If author does not exist, create new nested dictionary
#                 aggregated_data[author] = {subreddit: post_count}

# def process_directory(directory):
#     # Function to process each file in the directory
#     for filename in os.listdir(directory):
#         if filename.endswith('.tsv'):
#             file_path = os.path.join(directory, filename)
#             read_tsv_to_dict(file_path)

# # Main dictionary to hold all data
# aggregated_data = defaultdict(dict)

# # Path to the directory containing the TSV files
# directory_path = '/shared/3/projects/hiatus/tagged_data/full_pilot/long-reddit'
# process_directory(directory_path)

# # Now aggregated_data contains all the combined data
# print_stats(aggregated_data)


Total number of authors: 0
Total number of posts: 0
Average number of posts per author: 0.00
Total number of authors with over 5 total posts: 0
Total number of authors with over 10 total posts: 0
Total number of unique subreddits: 0


In [9]:
%%time
output_tsv = output_dir + file_name.split('.')[0] + '-counts.tsv'
# Write the post counts to a TSV file
# write_to_tsv(author_subreddit_counts, output_tsv)

CPU times: user 573 µs, sys: 366 µs, total: 939 µs
Wall time: 3.09 ms


## Join the partitioned files

In [ ]:
import os
import json
from glob import glob
from tqdm import tqdm

def join_tagged_files(input_directory, output_file, buffer_size=1024*1024):
    """Join all tagged files from the input directory into one output file with buffered writing."""
    tagged_files = glob(os.path.join(input_directory, "*.jsonl"))

    print(f"Total files to merge: {len(tagged_files)}")

    with open(output_file, 'w', buffering=buffer_size) as writer:
        for tagged_file in tqdm(tagged_files, desc="Merging tagged files"):
            with open(tagged_file, 'r') as reader:
                for line in reader:
                    try:
                        obj = json.loads(line)
                        writer.write(json.dumps(obj) + '\n')
                    except json.JSONDecodeError:
                        print(f"Error decoding JSON from file: {tagged_file}")


join_tagged_files(input_directory=tag_dir,
                  output_file=os.path.join(output_dir, file_name))

Total files to merge: 50



Merging tagged files:   0%|                                         | 0/50 [00:00<?, ?it/s]

## Delete the temp directories

In [11]:
# delete_partitioned_files(partition_dir)
# delete_partitioned_files(tag_dir)